## Data modeling

**1) Review Existing Unstructured Data and Diagram a New Structured Relational Data Model**

Review the 3 sample data files provided below. Develop a simplified, structured, relational diagram to represent how you would model the data in a data warehouse. The diagram should show each table’s fields and the joinable keys. You can use pencil and paper, readme, or any digital drawing or diagramming tool with which you are familiar. If you can upload the text, image, or diagram into a git repository and we can read it, we will review it!



Before creating a data model it would be helpful to visulaize the json data schema

In [154]:
import pandas as pd
import json
import numpy as np

df_receipts = pd.read_json("/content/data/receipts.json.gz", lines=True)
df_brands = pd.read_json("/content/data/brands.json.gz", lines=True)
df_users = pd.read_json("/content/data/users.json.gz", lines=True)

### Receipts Data

In [155]:
df_receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


Looks like column **rewardsReceiptItemList** contains a list of dictionaries. Let us explode the **rewardsReceiptItemList** into as many columns as the key value pairs

In [156]:
df_receipts = df_receipts.explode("rewardsReceiptItemList",ignore_index=True ) # explode column containing list of dicts
df_receipts = df_receipts.join(pd.json_normalize(df_receipts.rewardsReceiptItemList).add_prefix("ril_")).drop(columns=['rewardsReceiptItemList']) # add prefix ril_ (receipt item list) to indicate exploaded columns
df_receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,...,ril_itemNumber,ril_originalMetaBriteQuantityPurchased,ril_pointsEarned,ril_targetPrice,ril_competitiveProduct,ril_originalFinalPrice,ril_originalMetaBriteItemPrice,ril_deleted,ril_priceAfterCoupon,ril_metabriteCampaignId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [157]:
df_receipts.columns

Index(['_id', 'bonusPointsEarned', 'bonusPointsEarnedReason', 'createDate',
       'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate',
       'pointsEarned', 'purchaseDate', 'purchasedItemCount',
       'rewardsReceiptStatus', 'totalSpent', 'userId', 'ril_barcode',
       'ril_description', 'ril_finalPrice', 'ril_itemPrice',
       'ril_needsFetchReview', 'ril_partnerItemId',
       'ril_preventTargetGapPoints', 'ril_quantityPurchased',
       'ril_userFlaggedBarcode', 'ril_userFlaggedNewItem',
       'ril_userFlaggedPrice', 'ril_userFlaggedQuantity',
       'ril_needsFetchReviewReason', 'ril_pointsNotAwardedReason',
       'ril_pointsPayerId', 'ril_rewardsGroup', 'ril_rewardsProductPartnerId',
       'ril_userFlaggedDescription', 'ril_originalMetaBriteBarcode',
       'ril_originalMetaBriteDescription', 'ril_brandCode',
       'ril_competitorRewardsGroup', 'ril_discountedItemPrice',
       'ril_originalReceiptItemText', 'ril_itemNumber',
       'ril_originalMetaBriteQuan

#### Format columns to make it more readable, for dates convert into timestamp

In [158]:
#id
dfid = pd.json_normalize(df_receipts._id)
df_receipts['_id'] = dfid['$oid']


In [159]:
#createDate
df_cd = pd.json_normalize(df_receipts.createDate)
df_cd= df_cd.rename(columns={"$date":"createDate"})
df_receipts['createDate'] = df_cd['createDate']
df_receipts['createDate'] = pd.to_datetime(df_receipts.createDate, unit ='ms')

In [160]:
#dateScanned
df_ds = pd.json_normalize(df_receipts.dateScanned)
df_ds= df_ds.rename(columns={"$date":"dateScanned"})
df_receipts['dateScanned'] = df_ds['dateScanned']
df_receipts['dateScanned'] = pd.to_datetime(df_receipts.dateScanned, unit ='ms')

In [161]:
#finishedDate
df_fd = pd.json_normalize(df_receipts.finishedDate)
df_fd= df_fd.rename(columns={"$date":"finishedDate"})
df_receipts['finishedDate'] = df_fd['finishedDate']
df_receipts['finishedDate'] = pd.to_datetime(df_receipts.finishedDate, unit ='ms')

/usr/local/lib/python3.10/dist-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)


In [162]:
#modifyDate
df_md = pd.json_normalize(df_receipts.modifyDate)
df_md = df_md.rename(columns={"$date":"modifyDate"})
df_receipts['modifyDate'] = df_md['modifyDate']
df_receipts['modifyDate'] = pd.to_datetime(df_receipts.modifyDate, unit ='ms')

In [163]:
#pointsAwardedDate
df_pad = pd.json_normalize(df_receipts.pointsAwardedDate)
df_pad = df_pad.rename(columns={"$date":"pointsAwardedDate"})
df_receipts['pointsAwardedDate'] = df_pad['pointsAwardedDate']
df_receipts['pointsAwardedDate'] = pd.to_datetime(df_receipts.pointsAwardedDate, unit ='ms')

/usr/local/lib/python3.10/dist-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)


In [164]:
#purchaseDate
df_pd = pd.json_normalize(df_receipts.purchaseDate)
df_pd = df_pd.rename(columns={"$date":"purchaseDate"})
df_receipts['purchaseDate'] = df_pd['purchaseDate']
df_receipts['purchaseDate'] = pd.to_datetime(df_receipts.purchaseDate, unit ='ms')

/usr/local/lib/python3.10/dist-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)


In [165]:
df_receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,...,ril_itemNumber,ril_originalMetaBriteQuantityPurchased,ril_pointsEarned,ril_targetPrice,ril_competitiveProduct,ril_originalFinalPrice,ril_originalMetaBriteItemPrice,ril_deleted,ril_priceAfterCoupon,ril_metabriteCampaignId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,5.0,2021-01-03 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now, the data frame looks a lot cleaner and more readable. Let us clean column names and give _id a more meaningful name

In [166]:
df_receipts.columns

Index(['_id', 'bonusPointsEarned', 'bonusPointsEarnedReason', 'createDate',
       'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate',
       'pointsEarned', 'purchaseDate', 'purchasedItemCount',
       'rewardsReceiptStatus', 'totalSpent', 'userId', 'ril_barcode',
       'ril_description', 'ril_finalPrice', 'ril_itemPrice',
       'ril_needsFetchReview', 'ril_partnerItemId',
       'ril_preventTargetGapPoints', 'ril_quantityPurchased',
       'ril_userFlaggedBarcode', 'ril_userFlaggedNewItem',
       'ril_userFlaggedPrice', 'ril_userFlaggedQuantity',
       'ril_needsFetchReviewReason', 'ril_pointsNotAwardedReason',
       'ril_pointsPayerId', 'ril_rewardsGroup', 'ril_rewardsProductPartnerId',
       'ril_userFlaggedDescription', 'ril_originalMetaBriteBarcode',
       'ril_originalMetaBriteDescription', 'ril_brandCode',
       'ril_competitorRewardsGroup', 'ril_discountedItemPrice',
       'ril_originalReceiptItemText', 'ril_itemNumber',
       'ril_originalMetaBriteQuan

In [167]:
df_receipts=df_receipts.rename(columns={'ril_pointsEarned':'rilpointsEarned'})
df_receipts.columns = [col.replace('ril_', '') for col in df_receipts.columns]
df_receipts=df_receipts.rename(columns={'_id':'receipt_id'})

In [168]:
df_receipts.columns

Index(['receipt_id', 'bonusPointsEarned', 'bonusPointsEarnedReason',
       'createDate', 'dateScanned', 'finishedDate', 'modifyDate',
       'pointsAwardedDate', 'pointsEarned', 'purchaseDate',
       'purchasedItemCount', 'rewardsReceiptStatus', 'totalSpent', 'userId',
       'barcode', 'description', 'finalPrice', 'itemPrice', 'needsFetchReview',
       'partnerItemId', 'preventTargetGapPoints', 'quantityPurchased',
       'userFlaggedBarcode', 'userFlaggedNewItem', 'userFlaggedPrice',
       'userFlaggedQuantity', 'needsFetchReviewReason',
       'pointsNotAwardedReason', 'pointsPayerId', 'rewardsGroup',
       'rewardsProductPartnerId', 'userFlaggedDescription',
       'originalMetaBriteBarcode', 'originalMetaBriteDescription', 'brandCode',
       'competitorRewardsGroup', 'discountedItemPrice',
       'originalReceiptItemText', 'itemNumber',
       'originalMetaBriteQuantityPurchased', 'rilpointsEarned', 'targetPrice',
       'competitiveProduct', 'originalFinalPrice',
       'or

###  Users Data

In [169]:
df_users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


#### Format columns to make it more readable, for dates convert into timestamp

In [170]:
#id
dfid = pd.json_normalize(df_users._id)
df_users['_id'] = dfid['$oid']


In [171]:
#createdDate
df_cd = pd.json_normalize(df_users.createdDate)
df_cd = df_cd.rename(columns={"$date":"createdDate"})
df_users['createdDate'] = df_cd['createdDate']
df_users['createdDate'] = pd.to_datetime(df_users.createdDate, unit ='ms')

In [172]:
#lastLogin
df_ll = pd.json_normalize(df_users.lastLogin)
df_ll = df_ll.rename(columns={"$date":"lastLogin"})
df_users['lastLogin'] = df_ll['lastLogin']
df_users['lastLogin'] = pd.to_datetime(df_users.lastLogin, unit ='ms')

/usr/local/lib/python3.10/dist-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)


In [173]:
df_users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30.554,2021-01-03 15:25:30.596999936,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872,consumer,Email,WI


Now, the data frame looks a lot cleaner and more readable. Let us give _id a more meaningful name

In [174]:
df_users=df_users.rename(columns={'_id':'user_id'})
df_users.columns

Index(['user_id', 'active', 'createdDate', 'lastLogin', 'role', 'signUpSource',
       'state'],
      dtype='object')

### Brands Data

In [175]:
df_brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


Looks like column **cpg** contains a list of dictionaries. Let us explode the **cpg** into as many columns as the key value pairs

In [176]:
df_cpg = df_brands.explode("cpg",ignore_index=True ) # explode column containing list of dicts
df_brands = df_brands.join(pd.json_normalize(df_brands.cpg).add_prefix("cpg_")).drop(columns=['cpg']) # add prefix cpg_ to indicate exploaded columns
df_brands.head()

,_id,barcode,category,categoryCode,name,topBrand,brandCode,cpg_$ref,cpg_$id.$oid
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,test brand @1612366101024,0.0,NaN,Cogs,601ac114be37ce2ead437550
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,Starbucks,0.0,STARBUCKS,Cogs,5332f5fbe4b03c9a25efd0ba
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,Cogs,601ac142be37ce2ead437559
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,Cogs,601ac142be37ce2ead437559
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,Cogs,5332fa12e4b03c9a25efd1e7


#### Format columns to make it more readable

In [177]:
#id
dfid = pd.json_normalize(df_brands._id)
df_brands['_id'] = dfid['$oid']

In [178]:
df_brands.head()

,_id,barcode,category,categoryCode,name,topBrand,brandCode,cpg_$ref,cpg_$id.$oid
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,test brand @1612366101024,0.0,NaN,Cogs,601ac114be37ce2ead437550
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,Starbucks,0.0,STARBUCKS,Cogs,5332f5fbe4b03c9a25efd0ba
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,Cogs,601ac142be37ce2ead437559
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,Cogs,601ac142be37ce2ead437559
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,Cogs,5332fa12e4b03c9a25efd1e7


Now, the data frame looks a lot cleaner and more readable. Let us give _id a more meaningful name

In [179]:
df_brands=df_brands.rename(columns={'_id':'brand_id'})
df_brands.columns

Index(['brand_id', 'barcode', 'category', 'categoryCode', 'name', 'topBrand',
       'brandCode', 'cpg_$ref', 'cpg_$id.$oid'],
      dtype='object')

**Please refer fetch_rewards_data_model.pdf for the detailed relational data model**

## Load Data into tables

Let us load the dataframes according to the data model in a SQL database. I'm chosing sqlite. Please refer https://www.sqlite.org/lang.html for more info on the SQL syntax and dialect

#### Set up database

In [180]:
import sqlite3 as db


In [181]:
 #!pip install ipython-sql

In [182]:
conn = db.connect("fetch.db") #create a connection to a database

In [183]:
df_users.to_sql('dim_user',conn,if_exists='replace')

495

In [184]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [185]:
%sql sqlite:///fetch.db

In [186]:
%%sql

SELECT *
FROM dim_user limit 10

 * sqlite:///fetch.db
Done.


index,user_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,1,2021-01-03 15:24:04.800000,2021-01-03 15:25:37.857999,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,1,2021-01-03 15:24:04.800000,2021-01-03 15:25:37.857999,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,1,2021-01-03 15:24:04.800000,2021-01-03 15:25:37.857999,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,1,2021-01-03 15:25:30.554000,2021-01-03 15:25:30.596999,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,1,2021-01-03 15:24:04.800000,2021-01-03 15:25:37.857999,consumer,Email,WI
5,5ff1e194b6a9d73a3a9f1052,1,2021-01-03 15:24:04.800000,2021-01-03 15:25:37.857999,consumer,Email,WI
6,5ff1e1e8cfcf6c399c274ad9,1,2021-01-03 15:25:28.354000,2021-01-03 15:25:28.392000,consumer,Email,WI
7,5ff1e1b7cfcf6c399c274a5a,1,2021-01-03 15:24:39.626000,2021-01-03 15:24:39.664999,consumer,Email,WI
8,5ff1e194b6a9d73a3a9f1052,1,2021-01-03 15:24:04.800000,2021-01-03 15:25:37.857999,consumer,Email,WI
9,5ff1e1f1cfcf6c399c274b0b,1,2021-01-03 15:25:37.564000,2021-01-03 15:25:37.599000,consumer,Email,WI


In [187]:
df_brands.to_sql('dim_brand',conn,if_exists='replace')

1167

In [188]:
%%sql

SELECT *
FROM dim_brand limit 10

 * sqlite:///fetch.db
Done.


index,brand_id,barcode,category,categoryCode,name,topBrand,brandCode,cpg_$ref,cpg_$id.$oid
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,test brand @1612366101024,0.0,None,Cogs,601ac114be37ce2ead437550
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,Starbucks,0.0,STARBUCKS,Cogs,5332f5fbe4b03c9a25efd0ba
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,Cogs,601ac142be37ce2ead437559
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,Cogs,601ac142be37ce2ead437559
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,Cogs,5332fa12e4b03c9a25efd1e7
5,601ac142be37ce2ead43755b,511111719885,Baking,BAKING,test brand @1612366146091,0.0,TEST BRANDCODE @1612366146091,Cogs,601ac142be37ce2ead437559
6,601ac142be37ce2ead43755c,511111219897,Baking,BAKING,test brand @1612366146133,0.0,TEST BRANDCODE @1612366146133,Cogs,601ac142be37ce2ead437559
7,5cdad0f5166eb33eb7ce0faa,511111104810,Condiments & Sauces,None,J.L. Kraft,None,J.L. KRAFT,Cogs,559c2234e4b06aca36af13c6
8,5ab15636e4b0be0a89bb0b07,511111504412,Canned Goods & Soups,None,Campbell's Home Style,0.0,CAMPBELLS HOME STYLE,Cogs,5a734034e4b0d58f376be874
9,5c408e8bcd244a1fdb47aee7,511111504788,Baking,None,test,None,TEST,Cogs,59ba6f1ce4b092b29c167346


In [189]:
df_receipts.to_sql('fact_receipt',conn,if_exists='replace')

7381

In [192]:
%%sql

  SELECT *
  FROM fact_receipt limit 2

 * sqlite:///fetch.db
Done.


index,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,userFlaggedPrice,userFlaggedQuantity,needsFetchReviewReason,pointsNotAwardedReason,pointsPayerId,rewardsGroup,rewardsProductPartnerId,userFlaggedDescription,originalMetaBriteBarcode,originalMetaBriteDescription,brandCode,competitorRewardsGroup,discountedItemPrice,originalReceiptItemText,itemNumber,originalMetaBriteQuantityPurchased,rilpointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,FINISHED,26.0,5ff1e1eacfcf6c399c274ae6,4011,ITEM NOT FOUND,26.00,26.00,0,1,1,5.0,4011,1,26.00,5.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,FINISHED,11.0,5ff1e194b6a9d73a3a9f1052,4011,ITEM NOT FOUND,1,1,None,1,None,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


## SQL to answer predetermined business questions


**Write a query that directly answers a predetermined question from a business stakeholder**

Write a SQL query against your new structured relational data model that answers one of the following bullet points below of your choosing. Commit it to the git repository along with the rest of the exercise.

**1. What are the top 5 brands by receipts scanned for most recent month?**

In [209]:
%%sql

WITH receipts_cte as (
  SELECT receipt_id,
  dateScanned as date_scanned,
  strftime('%Y-%m', dateScanned)  as month_year_scanned,
  DENSE_RANK() OVER(ORDER BY strftime('%Y-%m', dateScanned) desc) AS rank_within_month_year,
  brandCode as receipt_brand_code
  FROM fact_receipt
  WHERE receipt_brand_code IS NOT NULL --- ASSUMPTION : ONLY CONSIDERING RECEIPTS WITH BRAND CODE PRESENT
  AND dateScanned  IS NOT NULL
),

receipts_cte_agg as (
  SELECT COUNT(DISTINCT receipt_id) as receipt_cnt,
  month_year_scanned,
  receipt_brand_code
  FROM receipts_cte
  WHERE
  receipts_cte.rank_within_month_year = 1 --most recent month
  GROUP BY month_year_scanned ,receipt_brand_code

),

brand_cte as (
  SELECT
  brandCode as brand_code,
  name as brand
  FROM dim_brand

)

SELECT ra.*,b.brand
FROM receipts_cte_agg ra
LEFT JOIN brand_cte b
ON ra.receipt_brand_code = b.brand_code
ORDER BY 1 DESC
limit 5

 * sqlite:///fetch.db
Done.


receipt_cnt,month_year_scanned,receipt_brand_code,brand
3,2021-02,BRAND,None
2,2021-02,MISSION,None
1,2021-02,VIVA,Viva


Explaination & Take aways :



*   The first CTE outputs recipts scanned and ranks the months from most recent to least recent. We are filtering for receipts where brandCode is not null. This is a data quality issue worth noting. Blank brandCode in receipt makes it impossible to tie reward to a brand resulting in a missed oppertunity.
*   The second CTE aggreagates receipt scanned count and the Third CTE provides brand name info. Blank brand name is another data qaulity issue
* The final select statement outputs the desired result



**2. How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?**

In [208]:
%%sql

WITH receipts_cte as (
  SELECT receipt_id,
  dateScanned as date_scanned,
  strftime('%Y-%m', dateScanned)  as month_year_scanned,
  DENSE_RANK() OVER(ORDER BY strftime('%Y-%m', dateScanned) desc) AS rank_within_month_year,
  brandCode as receipt_brand_code
  FROM fact_receipt
  WHERE receipt_brand_code IS NOT NULL --- ASSUMPTION : ONLY CONSIDERING RECEIPTS WITH BRAND CODE PRESENT
  AND dateScanned  IS NOT NULL
),

receipts_cte_agg as (
  SELECT COUNT(DISTINCT receipt_id) as receipt_cnt,
  month_year_scanned,
  receipt_brand_code
  FROM receipts_cte
  WHERE
  receipts_cte.rank_within_month_year = 2 --previous month
  GROUP BY month_year_scanned ,receipt_brand_code

),

brand_cte as (
  SELECT
  brandCode as brand_code,
  name as brand
  FROM dim_brand

)

SELECT ra.*,b.brand
FROM receipts_cte_agg ra
LEFT JOIN brand_cte b
ON ra.receipt_brand_code = b.brand_code
ORDER BY 1 DESC
limit 5


 * sqlite:///fetch.db
Done.


receipt_cnt,month_year_scanned,receipt_brand_code,brand
32,2021-01,BEN AND JERRYS,None
23,2021-01,FOLGERS,None
23,2021-01,PEPSI,Pepsi
22,2021-01,KELLOGG'S,None
22,2021-01,KRAFT,Kraft


Explaination & Take aways :

*   Similar logic applied as compared to previous query.
*   The top 5 brands are totally different and the counts of receipts scanned are a lot more



**3. When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?**

In [210]:
%%sql

with spend_cte as (
  SELECT receipt_id,
  totalSpent,
  rewardsReceiptStatus
  FROM fact_receipt
  WHERE (rewardsReceiptStatus LIKE "Accepted" OR rewardsReceiptStatus LIKE "Rejected")
  GROUP BY receipt_id,rewardsReceiptStatus
  ),

spend_agg_cte as (
    SELECT AVG(totalSpent) as average_spend,
    rewardsReceiptStatus
    FROM spend_cte
    GROUP BY rewardsReceiptStatus
  )

  SELECT *
  FROM spend_agg_cte

 * sqlite:///fetch.db
Done.


average_spend,rewardsReceiptStatus
23.326056338028184,REJECTED


In [213]:
%%sql

select
distinct (rewardsReceiptStatus)
from
fact_receipt


 * sqlite:///fetch.db
Done.


rewardsReceiptStatus
FINISHED
REJECTED
FLAGGED
SUBMITTED
PENDING


Explaination & Take aways :



*   The first query calculates average spend for 'Accepted’ or ‘Rejected’ reward receipt status
*   But based on the second query we find out there is no Accepted status in the datset. Hence a comparision between Accepted and Rejected status for average spend cannot be performed. However, should the business stakeholder require comparision between Rejected and other statuses like Finished, Pending etc. a small tweak in the query will yield desired comparision.



**4. When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?**

In [214]:
%%sql

with items_cte as (
  SELECT receipt_id,
  purchasedItemCount,
  rewardsReceiptStatus
  FROM fact_receipt
  WHERE (rewardsReceiptStatus LIKE "Accepted" OR rewardsReceiptStatus LIKE "Rejected")
  GROUP BY receipt_id,rewardsReceiptStatus
  ),

items_total_cte as (
    SELECT SUM(purchasedItemCount) as total_items,
    rewardsReceiptStatus
    FROM items_cte
    GROUP BY rewardsReceiptStatus
  )

  SELECT *
  FROM items_total_cte


 * sqlite:///fetch.db
Done.


total_items,rewardsReceiptStatus
173.0,REJECTED


In [215]:
%%sql

select
distinct (rewardsReceiptStatus)
from
fact_receipt

 * sqlite:///fetch.db
Done.


rewardsReceiptStatus
FINISHED
REJECTED
FLAGGED
SUBMITTED
PENDING


Explaination & Take aways :



*   The first query calculates total items purchased for 'Accepted’ or ‘Rejected’ reward receipt status
*   But based on the second query we find out there is no Accepted status in the datset. Hence a comparision between Accepted and Rejected status for total items purchased cannot be performed. However, should the business stakeholder require comparision between Rejected and other statuses like Finished, Pending etc. a small tweak in the query will yield desired comparision.


**5.Which brand has the most spend among users who were created within the past 6 months?**

In [216]:
%%sql

    SELECT  DATE(MAX(createdDate), '-6 months') as cut_off_date
    FROM dim_user



 * sqlite:///fetch.db
Done.


cut_off_date
2020-08-12


In [242]:
%%sql

with users_cte as(
  SELECT user_id ,
  createdDate	 as created_date
  FROM dim_user
  WHERE DATE(createdDate) >= '2020-08-12' -- users created in last 6 months
),
spend_cte as (
  SELECT receipt_id,
  userID,
  totalSpent,
  brandCode as receipt_brand_code
  FROM fact_receipt
  WHERE brandCode  NOT NULL --- ASSUMPTION
  GROUP BY 1,2,3,4
  ),
brand_cte as (
  SELECT
  brandCode as brand_code,
  name as brand
  FROM dim_brand
  WHERE brandCode  NOT NULL
)

Select
SUM(s.totalSpent) as total_spend,
b.brand_code,
b.brand
FROM
spend_cte s
LEFT JOIN
brand_cte b
ON s.receipt_brand_code  = b.brand_code
JOIN
users_cte u
ON s.userID = u.user_id
--WHERE b.brand_code <> 'None'
GROUP BY 2,3
ORDER BY 1 DESC
limit 2




 * sqlite:///fetch.db
Done.


total_spend,brand_code,brand
1314016.7199999937,None,None
25616.47,PEPSI,Pepsi


Explaination & Take aways :



*   The first query calculates the cut off date i.e 6 months before the most recent user created date.
*  The second query outputs the brand with the most spend amoung users created in the last 6 months which happens to be 'None'. The second highest spend is Pepsi. This is a data quality issue as we aren't able to attribute the highest spend to a brand.


**6.Which brand has the most transactions among users who were created within the past 6 months?**

In [247]:
%%sql

with users_cte as(
  SELECT user_id ,
  createdDate	 as created_date
  FROM dim_user
  WHERE DATE(createdDate) >= '2020-08-12' -- users created in last 6 months
),
tran_cte as (
  SELECT receipt_id,
  userID,
  brandCode as receipt_brand_code
  FROM fact_receipt
  WHERE brandCode  NOT NULL --- ASSUMPTION
  GROUP BY 1,2,3
  ),
brand_cte as (
  SELECT
  brandCode as brand_code,
  name as brand
  FROM dim_brand
  WHERE brandCode  NOT NULL
)

Select
COUNT(DISTINCT t.receipt_id) as total_transactions,
b.brand_code,
b.brand
FROM
tran_cte t
LEFT JOIN
brand_cte b
ON t.receipt_brand_code  = b.brand_code
JOIN
users_cte u
ON t.userID = u.user_id
--WHERE b.brand_code <> 'None'
GROUP BY 2,3
ORDER BY 1 DESC
limit 2


 * sqlite:///fetch.db
Done.


total_transactions,brand_code,brand
52,None,None
16,PEPSI,Pepsi


Explaination & Take aways :



*   The first query calculates the cut off date i.e 6 months before the most recent user created date.
*  The second query outputs the brand with the most transactions amoung users created in the last 6 months which happens to be 'None'. The second highest transaction is Pepsi, which is in line with the previous result. This is a data quality issue as we aren't able to attribute the highest transactions to a brand. Also an assumption is made that a transaction is identfied by a receipt_id.

## DATA Quality check


The python library ydata_profiling is used to perfrom a detailed data quality check and data profile of all tables. One of the objectives of data/analytics engineering is to produce high quality data sets which could be used for many use cases in addition to business insights like machine learning. Hence cooraltion plots and other statistical measures are included with the data profile report.

In [248]:
#!pip install ydata_profiling

#### Data Quality check for dim_users table

In [249]:
from ydata_profiling import ProfileReport
profile_users = ProfileReport(df_users, title="Users Profiling Report")
profile_users

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Key Data Quality issues


*   ~13% of lastLogin data is missing. This could hinder effetive marketing startegy execution.
*   ~11 % of State info is missing. This could hinder effetive marketing spend by state.
*   user_id is not distinct. Although not a data quality issue, it is woth to take note to ensure accurate querying/reporting






In [250]:
from ydata_profiling import ProfileReport
profile_brands= ProfileReport(df_brands, title="Brands Profiling Report")
profile_brands

Output hidden; open in https://colab.research.google.com to view.

Key Data Quality issues


*   ~20% of brandCode data is missing. This could hinder accurate attribution of sales and transactions to brands.
*   ~52 % of topBrand info is missing. This could hinder identifying top brands.
*   categoryCode and category have ~55% and ~13% data missing. This hinders the use of catagory code for marketing segmentation.


Similar analysis can be perfromed on the receipts data

In [252]:
# from ydata_profiling import ProfileReport
# profile_receipts= ProfileReport(df_receipts, title="Receipts Profiling Report")
# profile_receipts

## Communicate with Stakeholders

Please refer Fetch_challenge_Email_Slack_Message document for email to Business/Product leader